# Fundamentos de DataFrames en Spark

Los DataFrames de Spark son el componente principal y la forma más importante de trabajar con Spark y Python después de Spark 2.0. Los DataFrames actúan como versiones potentes de tablas, con filas y columnas, manejando fácilmente grandes conjuntos de datos. El cambio a DataFrames proporciona muchas ventajas:
* **Una sintaxis mucho más simple**
* **Capacidad de usar SQL directamente en el dataframe**
* **Las operaciones se distribuyen automáticamente entre RDDs**
    
Si has usado R o incluso la librería pandas con Python, probablemente ya estés familiarizado con el concepto de DataFrames. Los DataFrames de Spark amplían muchos de estos conceptos, permitiéndote transferir ese conocimiento fácilmente al comprender la sintaxis simple de los DataFrames de Spark. Recuerda que la ventaja principal de usar DataFrames de Spark versus esos otros programas es que **Spark puede manejar datos a través de muchos RDDs, conjuntos de datos enormes que nunca cabrían en una sola computadora**. Eso viene con un ligero costo de algunas opciones de sintaxis "peculiares", pero después de este curso te sentirás muy cómodo con todos esos temas.

¡Comencemos!

## Creando un DataFrame

Primero necesitamos iniciar una SparkSession:

In [ ]:
# Leemos un archivo JSON desde el sistema de archivos de Databricks (DBFS)
# spark.read.json() es el método para leer archivos en formato JSON
# "dbfs:/" indica que el archivo está en el sistema de archivos distribuido de Databricks
df = spark.read.json("dbfs:/databricks-datasets/structured-streaming/events/file-0.json")

Luego iniciamos la SparkSession

In [ ]:
# Comentado: dbutils.fs.ls ("FileStore/tables") permitiría listar archivos en esa ruta
# type(df) nos devuelve el tipo de objeto que es 'df'
# Verificamos que efectivamente df es un DataFrame de PySpark
type(df)

Primero necesitarás obtener los datos de un archivo (o conectarte a un archivo distribuido grande como HDFS, hablaremos de esto más adelante una vez que pasemos a conjuntos de datos más grandes en AWS EC2).

#### Mostrando los datos

In [ ]:
# df.show() muestra las primeras 20 filas del DataFrame en formato tabla
# Es el equivalente a head() en pandas, pero por defecto muestra 20 filas
# Muy útil para inspeccionar rápidamente los datos y su estructura
df.show()

In [ ]:
# df.printSchema() imprime el esquema (estructura) del DataFrame
# Muestra el nombre de cada columna, su tipo de dato y si puede ser nulo
# Es fundamental para entender qué tipo de datos tenemos en cada columna
df.printSchema()

In [ ]:
# df.columns devuelve una lista con los nombres de todas las columnas del DataFrame
# Es útil cuando queremos ver rápidamente qué columnas tenemos disponibles
# El resultado es una lista de Python que podemos usar en otros procesos
df.columns

In [ ]:
# df.describe() genera estadísticas descriptivas de las columnas numéricas
# Incluye: count (conteo), mean (media), stddev (desviación estándar), min y max
# Devuelve un DataFrame, no imprime directamente (usa .show() para visualizar)
df.describe()

Algunos tipos de datos facilitan la inferencia del esquema (como formatos tabulares como csv que mostraremos más adelante). 

Sin embargo, a menudo tienes que establecer el esquema tú mismo si no estás tratando con un método .read que no tiene inferSchema() incorporado.

Spark tiene todas las herramientas que necesitas para esto, solo requiere una estructura muy específica:

In [ ]:
# Importamos las clases necesarias para definir esquemas manualmente
# StructField: define un campo individual (columna) con nombre, tipo y nulabilidad
# StringType: tipo de dato para cadenas de texto
# LongType: tipo de dato para números enteros largos
# StructType: define la estructura completa del DataFrame (conjunto de campos)
from pyspark.sql.types import StructField, StringType, LongType, StructType

A continuación necesitamos crear la lista de campos de estructura
* Parámetro name: string, nombre del campo.
* Parámetro dataType: :class:`DataType` del campo.
* Parámetro nullable: booleano, si el campo puede ser nulo (None) o no.

In [ ]:
# Definimos el esquema como una lista de StructField
# Cada StructField especifica: nombre de columna, tipo de dato, y si acepta nulos
# "time" será tipo Long (entero largo) y puede ser nulo (True)
# "action" será tipo String (texto) y puede ser nulo (True)
data_schema = [StructField("time", LongType(), True),
              StructField("action", StringType(), True)]

In [ ]:
# Creamos el objeto StructType que contiene todos los campos del esquema
# Este es el objeto final que representa la estructura completa del DataFrame
# Lo usaremos como parámetro 'schema' al leer datos para forzar esta estructura
final_struc = StructType(fields=data_schema)

In [ ]:
# Ahora leemos el JSON pero especificando nuestro esquema predefinido
# schema=final_struc le dice a Spark exactamente qué estructura esperar
# Esto es más eficiente que inferir el esquema y evita errores de interpretación
df = spark.read.json("dbfs:/databricks-datasets/structured-streaming/events/file-0.json", schema = final_struc)

In [ ]:
# Imprimimos el esquema para verificar que se aplicó correctamente
# Ahora no hay necesidad de inferir tipos, Spark usa directamente lo que especificamos
# Esto hace que la lectura sea más rápida y predecible
df.printSchema()

### Obteniendo los datos

In [ ]:
# Accedemos a una columna específica usando notación de corchetes
# df["time"] devuelve un objeto Column, NO un DataFrame
# Es similar a acceder a una columna en pandas, pero el comportamiento es diferente
df["time"]

In [ ]:
# Verificamos el tipo del objeto que devuelve df["time"]
# Confirma que es un objeto Column de PySpark
# Las columnas se usan en operaciones de filtrado y transformación
type(df["time"])

In [ ]:
# df.select() selecciona columnas específicas y devuelve un nuevo DataFrame
# A diferencia de df["time"], select() SIEMPRE devuelve un DataFrame
# Es la forma recomendada de seleccionar columnas en PySpark
df.select("time")

In [ ]:
# Confirmamos que select() devuelve un DataFrame, no una Column
# Esto es importante porque los DataFrames tienen métodos diferentes a las Column
# Por ejemplo, solo los DataFrames tienen el método .show()
type(df.select("time"))

In [ ]:
# Seleccionamos la columna "time" y mostramos el resultado en formato tabla
# Este es el flujo típico: select() para crear un nuevo DataFrame y .show() para visualizarlo
# Muestra las primeras 20 filas de la columna seleccionada
df.select("time").show()

In [ ]:
# df.head(n) devuelve las primeras n filas como una lista de objetos Row
# Aquí pedimos las primeras 2 filas del DataFrame completo
# Cada Row contiene todos los valores de esa fila, accesibles por nombre de columna
df.head(2)

In [ ]:
# Seleccionamos TODAS las columnas del DataFrame usando df.columns
# df.columns devuelve una lista con los nombres, y select() puede recibir una lista
# Es útil cuando quieres seleccionar todas las columnas dinámicamente
df.select(df.columns).show()

Múltiples columnas:

In [ ]:
# select() puede recibir múltiples nombres de columnas como argumentos separados
# Esto crea un nuevo DataFrame con solo las columnas "time" y "action"
# No se modifica el DataFrame original, se crea uno nuevo
df.select("time", "action")

In [ ]:
# Seleccionamos múltiples columnas y mostramos el resultado
# Es la forma más común de trabajar: seleccionar las columnas necesarias y visualizarlas
# Útil para exploración de datos y análisis inicial
df.select("time", "action").show()

### Creando nuevas columnas

In [ ]:
# withColumn() crea una nueva columna o modifica una existente
# Primer argumento: nombre de la nueva columna ("newtime")
# Segundo argumento: expresión para calcular los valores (aquí sumamos 5 a "time")
# Esto crea un nuevo DataFrame con todas las columnas originales más "newtime"
df.withColumn("newtime", df["time"] + 5).show()

In [ ]:
# Mostramos el DataFrame original para demostrar que NO se ha modificado
# En PySpark, las transformaciones son inmutables: crean nuevos DataFrames
# Para conservar los cambios, debemos reasignar: df = df.withColumn(...)
df.show()

In [ ]:
# withColumnRenamed() renombra una columna existente
# Primer argumento: nombre actual de la columna ("action")
# Segundo argumento: nuevo nombre para la columna ("superaction")
# También es una operación inmutable, devuelve un nuevo DataFrame
df.withColumnRenamed("action", "superaction").show()

Operaciones más complicadas para crear nuevas columnas

In [ ]:
# Creamos una nueva columna multiplicando los valores de "time" por 2
# Podemos usar operadores aritméticos directamente sobre columnas
# show(5) limita la visualización a solo 5 filas en lugar de las 20 por defecto
df.withColumn("doubletime", df["time"]*2).show(5)

In [ ]:
# Creamos una columna sumando 1 a cada valor de "time"
# Las operaciones aritméticas básicas (+, -, *, /) funcionan directamente
# Spark maneja automáticamente la aplicación de la operación a todas las filas
df.withColumn("add_one_time", df["time"] + 1).show()

In [ ]:
# Dividimos los valores de "time" por 2 para crear "half_time"
# La división produce números decimales (tipo double en PySpark)
# Nota la notación científica en los resultados (7.34750E8 = 734,750,000)
df.withColumn("half_time", df["time"]/2).show(5)

In [ ]:
# Sin .show(), withColumn() solo devuelve el objeto DataFrame
# Aquí vemos que el DataFrame resultante tiene 3 columnas: time, action, half_time
# Y "half_time" es de tipo double (número decimal de doble precisión)
df.withColumn("half_time", df["time"]/2)

¡Discutiremos operaciones mucho más complicadas más adelante!

### Usando SQL

Para usar consultas SQL directamente con el dataframe, necesitarás registrarlo en una vista temporal:

In [ ]:
# createOrReplaceTempView() registra el DataFrame como una tabla temporal
# Esto nos permite usar consultas SQL sobre el DataFrame
# "IoT" es el nombre que le damos a la vista temporal (puede ser cualquier nombre)
# La vista existe solo durante la sesión de Spark actual
df.createOrReplaceTempView("IoT")

In [ ]:
# spark.sql() ejecuta una consulta SQL y devuelve un DataFrame
# Aquí usamos SELECT * para obtener todas las columnas de la vista "IoT"
# Es útil si ya conoces SQL y prefieres esa sintaxis sobre los métodos de DataFrame
sql_results = spark.sql("SELECT * FROM IoT")

In [ ]:
# Mostramos los resultados de la consulta SQL
# El resultado es idéntico a hacer df.show()
# Puedes usar cualquier consulta SQL válida: WHERE, GROUP BY, JOIN, etc.
sql_results.show()

No nos enfocaremos realmente en usar la sintaxis SQL para este curso en general, pero ten en cuenta que siempre está ahí para ayudarte a salir rápidamente de un apuro con tus habilidades de SQL.

¡Muy bien, eso es todo lo que necesitamos saber por ahora!